In [1]:
import numpy as np
import pickle
import os
import pandas as pd
import json

### Weights Loading

Choose your path and load the .pkl file with weights matrices. Also choose the name for your data.

In [2]:
# load weights
pathToWeights = r"C:\Users\Theodore\Dropbox\ExampleData\Opt_rmsprop__Dro_0.0__Num_20_20_16__Wei_0_pruned_wts.pkl"
# Choose the suffix of your data name. Default are FeedforwardNN. This name string will be added after it.
name = ''

# read in weight data
wts =  pickle.load(open(pathToWeights, 'rb'))

# print sizes of each weight matrix
wtLengths = []
max_width = 0
for ii in range(len(wts)):
    print(wts[ii].shape)
    wtLengths.append(np.prod(wts[ii].shape))
    max_width = max(max_width, wts[ii].shape[0])

# print total number of weights (including biases)
print(np.sum(wtLengths), "total weights")

(10, 20)
(20,)
(20, 20)
(20,)
(20, 16)
(16,)
(16, 7)
(7,)
1095 total weights


In [3]:
# Specify upper limit for maximal layer size
# This is converting the number to a power of 10, making it convenient for node indexing
MaxSize = int(10**np.ceil(np.log(max_width)/np.log(10)))

### Graph of Neural Network

Constructing a json file with the graph structure of the neural network (nodes and links). This part is for direct visualization of the static neural network structure with weights. Not used to calculate the node values interactively.

In [4]:
nodes = []
links = []
# iterate through the layers to get links and nodes
for layerNum in np.arange(0, len(wts), 2):
    m, n = wts[layerNum].shape
    for i in range(m):
        node_id = int(layerNum/2*MaxSize + i)
        nodes.append({'id':node_id, 'LayerNum':int(layerNum/2), 'NodeNum':i})
        for j in range(n):
            source = node_id
            target = int((layerNum/2+1)*MaxSize + j)
            links.append({'source':source, 'target':target, 'weight':float(wts[layerNum][i,j])})
# output layer nodes
for l in range(len(wts[-1])):
    node_id = int(len(wts)/2*MaxSize + l)
    nodes.append({'id':node_id, 'LayerNum':int(len(wts)/2), 'NodeNum':i})
# Build the network graph
Graph = {'nodes': nodes, 'links': links}
with open('NetworkGraph'+name+'.json', 'w') as fp:
    json.dump(Graph, fp)             

### Weight Tensors in Json

Import the weights and biases to json. This is the main data structure used in the interactive visualization. The neural network graph and node computation are done in the code in JavaScript after importing this.

In [5]:
Weights = []
Bias = []
for l in np.arange(0, len(wts), 2):
    W = []
    for i in range(wts[l].shape[0]):
        w = []
        for j in range(wts[l].shape[1]):
            w.append(float(wts[l][i,j]))
        W.append(w)
    Weights.append(W)
    b = []
    for i in range(len(wts[l+1])):
        b.append(float(wts[l+1][i]))
    Bias.append(b)
FeedforwardNN = {'weights': Weights, 'bias': Bias, 'WidthLimit': MaxSize}
with open('FeedforwardNN'+name+'.json', 'w') as fp:
    json.dump(FeedforwardNN, fp)      